In [1]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [2]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [5]:
url_star = 'https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALlmsecBsACAdICJDFmMDhmM2QxLTMxYTAtNDMzMC05ODYwLTllNWU2M2EyMDZiN9gCBeACAQ&aid=304142&ss=Amsterdam&ssne=Vienna%2C+Austria&ssne_untouched=Vienna%2C+Austria&efdco=1&lang=en-us&src=searchresults&dest_id=-2140479&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=8dd8525fef6400c2&ac_meta=GhA4ZGQ4NTI1ZmVmNjQwMGMyIAAoATICZW46CUFtc3RlcmRhbUAASgBQAA%3D%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bclass%3D'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + '&order=distance_from_search')

In [6]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(6):
    time.sleep(3)  
    if star == 0:
        
        for i in range(0, 126, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
        time.sleep(3) 
    if star == 1:
        for i in range(0, 26, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
        time.sleep(3)
    


print(len(urls_hotel_4))
print(len(urls_hotel_5))

134
26


In [7]:
urls_hotel_4_en = []
urls_hotel_5_en = []

    
for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

134
26


In [8]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [88]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_amsterdam = pd.DataFrame(data=data_array_full, columns=columns)
hotel_amsterdam.replace('nan', np.NaN, inplace=True)
hotel_amsterdam = hotel_amsterdam.dropna(thresh=2).reset_index(drop=True)
hotel_amsterdam

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\nDamrak 96, Amsterdam City Center, 1012 LP Am...","1,896 reviews",Excellent
2,4,"\nDam 9, Amsterdam City Center, 1012 JS Amster...",181 reviews,Very Good
3,4,"\nDamrak 93-94, Amsterdam City Center, 1012 LP...","1,799 reviews",Good
4,4,"\nNieuwezijds Voorburgwal 176 - 180, Amsterdam...","1,728 reviews",Very Good
...,...,...,...,...
156,5,"\nAmsteldijk 67, Oud Zuid, 1074 HZ Amsterdam, ...","3,664 reviews",Excellent
157,5,"\nFerdinand Bolstraat 333, Oud Zuid, 1072 LH A...",864 reviews,Excellent
158,5,"\nDijsselhofplantsoen 7, Oud Zuid, 1077 BJ Ams...","1,065 reviews",Good
159,5,"\nApollolaan 138, Oud Zuid, 1077 BG Amsterdam,...",655 reviews,Very Good


In [90]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [91]:
hotel_amsterdam['hotel_address'] = hotel_amsterdam['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).strip().lower().replace(' ', ''))

In [96]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().replace(' ', ''))

In [98]:
hotel_df = hotel_df.merge(
                hotel_amsterdam,
                on='hotel_address',
                how='left')

In [99]:
null = hotel_df[hotel_df['city_hotel'] == 'Amsterdam'].isnull().mean() * 100
null_per = null[null > 0]
null_per

hotel_star       20.532019
reviews_count    20.532019
quality          20.532019
dtype: float64

In [92]:
dupl_columns = list(hotel_amsterdam.columns) 

mask = hotel_amsterdam.duplicated(subset=dupl_columns) 
hotel_duplicates = hotel_amsterdam[mask]
print(f'Число найденных дубликатов: {hotel_duplicates.shape[0]}')

Число найденных дубликатов: 9


In [93]:
hotel_amsterdam = hotel_amsterdam.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {hotel_amsterdam.shape[0]}')

Результирующее число записей: 152


In [101]:
hotel_amsterdam.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_amsterdam.csv')